In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/Users/apple/Desktop/Kaggle-Competition-Titanic/models')
from collections import Counter
import kmodes

In [3]:
pd.read_csv('train_processed.csv', usecols=range(1,9)).head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,1,1,0,0,2
1,1,1,1,2,1,0,3,0
2,1,3,1,1,0,0,1,2
3,1,1,1,2,1,0,3,2
4,0,3,0,2,0,0,1,2


In [ ]:
np.argsort(lst)[-1]

In [5]:
# kmodes Model with KNN
data = 'train_processed.csv'
X = kmodes.get_data(data)
# print(X[:4])
num_instances = len(X)
# store the best records in 5 iterations 
best_score = 0
best_centroids = []
best_clusters =[]
best_iteratoin = 0
for i in range(5):
    centroids, clusters , iteration= kmodes.find_centers(X, 2)
    purity = kmodes.get_purity(clusters, centroids, num_instances)
    if purity > best_score:
        best_centroids = centroids
        best_clusters = clusters
        best_score = purity
        best_iteratoin = iteration
# centroids, clusters, iteration= kmodes.find_centers(X, 2)
# purity = kmodes.get_purity(clusters, centroids, num_instances)
# for k in best_clusters.keys():
#         points = np.array(best_clusters[k])
#         class_attr = Counter(points[:, -1]).most_common(1)
#         print(class_attr)
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str).values.flatten()
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9),dtype=str).values.tolist()
for idx, c in enumerate(best_centroids):
    labels = np.array(best_clusters[idx])[:,-1]
    label = Counter(labels).most_common(1)[0][0]
    c[-1] = label
# print(best_centroids)
count = 0
for i, x in enumerate(test_data):
    distance = [kmodes.get_distance(x, c[:-1]) for c in best_centroids]
    idx = np.argsort(distance)[0]
    label = best_centroids[idx][-1]
#     print(actual[i])
#     print('%s vs %s' % (label, actual[i]))
    if str(label) == str(actual[i]):
        count += 1
acc_kmode_knn = round(float(count)/len(actual)*100, 2)
print('The best accuracy for the task is %s' % acc_kmode_knn)

IOError: File data/gender_submission.csv does not exist

In [61]:
# Decision Tree using Gain Ratio
import decisionTree_GR
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
# print(train_data.head())
test_data = pd.concat([actual, test_data], axis=1)
# print(test_data.head())
actual = actual.values.flatten()
rows = train_data.values.tolist()
# print(rows[:2])
test_rows = test_data.values.tolist()
root = decisionTree_GR.buildtree(rows)
# decisionTree_GR.prune(root, 0.1)
predictions = []
# print(actual)
for row in test_rows:
    predicted = decisionTree_GR.classify(row, root).most_common(1)[0][0]
    predictions.append(predicted)
# print(predictions)
acc_decisionTree_GR = round(decisionTree_GR.evaluate_accuracy(actual, predictions)*100, 2)
print('The best accuracy for C4.5 decision tree is %s' % acc_decisionTree_GR)

The best accuracy for C4.5 decision tree is 81.58


In [65]:
# Naive Bayers 
import NaiveBayers
from collections import defaultdict
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
test_data = pd.concat([actual, test_data], axis=1)
actual = actual.values.flatten()
rows = train_data.values.tolist()
test_rows = test_data.values.tolist()
classes = defaultdict(int)
# I create one more attribute that is unnecessary because I want to index to correspond to attributes
# attributes: list of dicts: key is 'classValue + attributeValue', value is its corresponding count
attributes = [defaultdict(int) for i in range(len(rows[0]))]
NaiveBayers.prepare(classes, attributes, rows)
acc_NaiveBayers = round(NaiveBayers.test(classes, attributes, test_rows, actual)*100, 2)
print('The accuracy of Naive Bayers is %s' % acc_NaiveBayers)       

The accuracy of Naive Bayers is 94.02


In [3]:
# Multi-perceptron
import multi_perceptron
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
# X = list of (label, dict)


In [75]:
models = pd.DataFrame({
    'Model': ['kmodes with KNN', 'Multinomial Naive Bayers', 
              'Decision Tree'],
    'Score': [acc_kmode_knn, acc_decisionTree_GR, acc_NaiveBayers]})
models.sort_values(by='Score', ascending=False)

,Model,Score
2,Decision Tree,94.02
0,kmodes with KNN,82.30
1,Multinomial Naive Bayers,81.58
